In [ ]:
import vispy
# selecting sub-backend behind jupyter_rfb:
# 1) pyglfw has high priority if present
# 2) to use any other backend, install then designate as default
# 4) use jupyter_rfb, it then selects the sub-backend according to config
# 5) enable jupyterlab extensions for interactive widget provided by jupyter_rfb

# only needed in jhub2, jhub and jlab-hpc can't run opengl at all, locally auto-selected backend is OK
vispy.config.update(default_backend='egl')
vispy.use('jupyter_rfb')

In [ ]:
import numpy as np
import pandas as pd
import vispy
from matplotlib import pyplot as plt

from datasets import get_hits
from tracking.plot import plot_event, plot_seg_diff, _seg_tseg_view
from hopfield.plot import plot_result, _result_view
from segment.track import gen_seg_track_layered
from segment.candidate import gen_seg_layered
from hopfield.energy import energy
from hopfield.energy.cross import cross_energy_matrix
from hopfield.energy.curvature import curvature_energy_matrix, segment_adjacent_pairs
from hopfield.iterate import annealing_curve, hopfield_history, gen_perfect_act

In [ ]:
N_EVENTS = 200
NOISES = (0, 100, 50*35)
config_100={'alpha': 19.6857149529666, 'bias': -0.31150606683750404, 'cooling_steps': 9, 'cosine_min_allowed': -0.9929937414803206,
            'cosine_min_rewarded': 0.09703011610247317, 'cosine_power': 11.6462959878647, 'distance_power': 0.19013229061815848, 'dropout': 0,
            'gamma': 12.528629082393694, 'initial_act': 0.09188054593810106, 'learning_rate': 0.2964621247552072, 'rest_steps': 0,
            't_max': 6.986302071223637, 't_min': 1.0, 'threshold': 0.5}
config_50x={'alpha': 4.766456909730726, 'bias': -1.5899727982927079, 'cooling_steps': 53, 'cosine_min_allowed': -0.37559678469140234,
           'cosine_min_rewarded': 0.9977232547138374, 'cosine_power': 16.12619322582289, 'distance_power': 0.1481960766610274,
           'dropout': 0, 'gamma': 9.827156643218887, 'initial_act': 0.7822536857072754, 'learning_rate': 0.41337726498711364,
           'rest_steps': 14, 't_max': 45.4967596622952, 't_min': 1.0, 'threshold': 0.5}
CONFIGS = (config_100, config_100, config_50x)

In [ ]:
full_events = get_hits('spdsim', N_EVENTS, n_noise_hits=max(NOISES))
full_events

In [ ]:
I = 1
N_NOISE = NOISES[I]
config = CONFIGS[I]
locals().update(config)

In [ ]:
def lower_noise(event, n_noise_hits):
    return pd.concat([event[event.track != -1],
                       event[event.track == -1].head(n_noise_hits)])

events = full_events.groupby('event_id', group_keys=False).apply(lower_noise, N_NOISE)
event = events[events.event_id==0]
event

In [ ]:
seg_canvas = vispy.scene.SceneCanvas(bgcolor='white', size=(200, 500))
grid = seg_canvas.central_widget.add_grid()
grid.add_widget(_seg_tseg_view(event, gen_seg_layered(event), gen_seg_track_layered(event),
                              camera=vispy.scene.cameras.TurntableCamera(fov=20, scale_factor=2000)
                              ), row=0, col=0)
seg_canvas

In [ ]:
vispy.io.image.imsave(f'seg_{N_NOISE}_noise.png', seg_canvas.render())

In [ ]:
pos = event[['x', 'y', 'z']].to_numpy()
seg = gen_seg_layered(event)
pairs = segment_adjacent_pairs(seg)
crossing_matrix = cross_energy_matrix(seg)
curvature_matrix = curvature_energy_matrix(
    pos, seg, pairs, alpha=alpha, gamma=gamma,
    cosine_threshold=cosine_min_rewarded, cosine_min_allowed=cosine_min_allowed,
    curvature_cosine_power=cosine_power,
    distance_prod_power_in_denominator=distance_power
)
energy_matrix = crossing_matrix + curvature_matrix
temp_curve = annealing_curve(t_min, t_max, cooling_steps, rest_steps)
starting_act = np.full(len(seg), initial_act)
acts = hopfield_history(energy_matrix, temp_curve, starting_act, dropout, learning_rate, bias)

In [ ]:
positive = [act >= threshold for act in acts]

In [ ]:
tseg = gen_seg_track_layered(event)
len(seg), len(tseg)

In [ ]:
event_canvas = plot_event(event, tseg, fig_size=(900, 750),
                         camera=vispy.scene.cameras.TurntableCamera(fov=0, scale_factor=2000))
event_canvas

In [ ]:
vispy.io.image.imsave(f'event_{N_NOISE}_noise.png', event_canvas.render())

In [ ]:
perfect_act = gen_perfect_act(seg, tseg)
perfect_act

In [ ]:
tcdf = pd.DataFrame({'temp': temp_curve})
tcdf.index.name = 'step'
tcdf.plot()
plt.savefig(f'temp_{N_NOISE}_noise_{N_EVENTS}_events.png')

In [ ]:
energy_history = pd.DataFrame({
    'energy': [energy(energy_matrix, act) for act in acts],
    'curvature_energy': [energy(curvature_matrix, act) for act in acts],
    'crossing_energy': [energy(crossing_matrix, act) for act in acts]
})
energy_history.index.name = 'step'
energy_history.energy = 1 + energy_history.energy - energy_history.energy.min()
energy_history.plot(logy=True)
plt.show()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
perfect_act = gen_perfect_act(seg, tseg)
small_history = pd.DataFrame([
    (
        precision_score(perfect_act, act > config['threshold'], zero_division=0),
        recall_score(perfect_act, act > config['threshold'], zero_division=0),
        f1_score(perfect_act, act > config['threshold'], zero_division=0),
        roc_auc_score(perfect_act, act),
        average_precision_score(perfect_act, act),
        act.mean(),
        ((act - perfect_act) ** 2).mean()
    ) for act in acts],
    columns=['precision', 'recall', 'f1', 'roc_auc', 'ap', 'mean_act', 'dist_perfect'])
small_history.index.name = 'step'
small_history.plot()
plt.show()

In [ ]:
from sklearn.metrics import precision_recall_curve, PrecisionRecallDisplay, average_precision_score

act = acts[-1]
average_precision = average_precision_score(perfect_act, act)
precision, recall, _ = precision_recall_curve(perfect_act, act)
PrecisionRecallDisplay(precision=precision, recall=recall, average_precision=average_precision,
                       estimator_name='hopfield').plot()
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, RocCurveDisplay

fpr, tpr, _ = roc_curve(perfect_act, act)
RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc_score(perfect_act, act), estimator_name='hopfield').plot()
plt.show()

In [ ]:
from sklearn.metrics import det_curve, DetCurveDisplay

fpr, fnr, _ = det_curve(perfect_act, act)
DetCurveDisplay(fpr=fpr, fnr=fnr).plot()
plt.show()

In [ ]:
result_canvas = vispy.scene.SceneCanvas(bgcolor='white', size=(300, 750))
positives = [act >= threshold for act in acts]
result_canvas.central_widget.add_widget(
    _result_view(event, seg, acts[-1], perfect_act, positives[-1], 
                 camera=vispy.scene.cameras.TurntableCamera(fov=20, scale_factor=2000))
)    
result_canvas

In [ ]:
vispy.io.image.imsave(f'result_{N_NOISE}_noise.png', result_canvas.render())

In [ ]:
from datasets import get_hits
from hopfield import iterate
from metrics.tracks import track_metrics
from tqdm import tqdm
tqdm.pandas()

def evaluate(event):
    event = event.reset_index(drop=True)
    seg, acts, positives = iterate.run(event, **config)
    tseg = gen_seg_track_layered(event)
    score = track_metrics(event, seg, tseg, acts[-1], positives[-1])
    score['total_steps'] = config['cooling_steps'] + config['rest_steps']
    score['trackml_loss'] = 1. - score['trackml']
    return pd.Series(score, dtype=object)

events = get_hits('spdsim', N_EVENTS, n_noise_hits=N_NOISE)
stats = events.groupby('event_id').progress_apply(evaluate)
stats


In [ ]:
stats.hist('n_fp_seg', bins=20, range=(0,40))
plt.savefig(f'n_fp_{N_NOISE}_noise_{N_EVENTS}_events.png')

In [ ]:
stats.hist('trackml', bins=20, range=(0,1))
plt.savefig(f'trackml_{N_NOISE}_noise_{N_EVENTS}_events.png')

In [ ]:
# stats.hist('false tracks', bins=20)
# plt.savefig(f'false_{N_NOISE}_noise_{N_EVENTS}_events.png')

In [ ]:
# stats.hist('missed tracks', bins=20)
# plt.savefig(f'missed_{N_NOISE}_noise_{N_EVENTS}_events.png')

In [ ]:
# import seaborn as sns
# sns.relplot(stats, x='trackml_loss', y='false positive segments',
#             hue='missed tracks', palette=sns.color_palette(),
#             style='false tracks')
# plt.savefig(f'rel_{N_NOISE}_noise_{N_EVENTS}_events.png')

In [ ]:
# import seaborn as sns
# sns.catplot(stats, kind='strip', orient='h',
#             x='trackml score', y='false tracks',
#             hue='missed tracks', palette=sns.color_palette())
# plt.savefig(f'cat_{N_NOISE}_noise_{N_EVENTS}_events.png')

In [ ]:
# (stats['missed tracks']==0).sum() / stats['missed tracks'].count()